<img width="10%" alt="Naas" src="https://landen.imgix.net/jtci2pxwjczr/assets/5ice39g4.png?w=160"/>

# Snowflake - JSON data ingestion from local stage

<a href="https://app.naas.ai/user-redirect/naas/downloader?url=https://raw.githubusercontent.com/jupyter-naas/awesome-notebooks/master/Snowflake/Snowflake_Ingest_json_data_from_local_stage.ipynb" target="_parent"><img src="https://naasai-public.s3.eu-west-3.amazonaws.com/open_in_naas.svg"/></a>

**Tags:** #snowflake #data #warehouse #naas_drivers #snippet

**Author:** [Mateusz Polakowski](https://www.linkedin.com/in/polakowski/)

To use your own data inside Snowflake DWH, first thing first, you need to ingest it.

This notebook shows how to put <u>JSON</u> data from your local machine into Snowflake. Several objects and actions will be necessary to do so:

- creating file format,
- creating internal named stage (for other stages, please check [Snowflake documentation](https://docs.snowflake.com/en/user-guide/data-load-local-file-system-create-stage.html)),
- putting file into stage,
- copying data from stage to the table.

All above steps are essential to make use of the data from your local machine inside Snowflake.

You can try with your own JSON data, but to follow the notebook cells 1:1 please download `reviews_dev.json` file from `awesome-notebooks/Snowflake` directory. Eventually, you may need to change your file location below in the ensuing cells.

## Input

### Import library

In [1]:
import os
from naas_drivers import snowflake
from snowflake.connector.errors import ProgrammingError

### Setup Snowflake account

If you don't have your SF account, you can easily set up a [30-day trial account with $400 budget here](https://signup.snowflake.com/).

### Credentials

In [2]:
# Here environment variables are used to pass Snowflake credentials, 
# but it's okay to do it in a different manner

sf_username=os.environ['SNOWFLAKE_USER']
sf_password=os.environ['SNOWFLAKE_PASSWORD']
sf_account=os.environ['SNOWFLAKE_ACCOUNT']

## Model

### Connecting to your Snowflake account

In [3]:
snowflake.connect(
    username=sf_username,
    password=sf_password,
    account=sf_account
)

### Environment setup

In [4]:
snowflake.database.get_current() is None

True

In [5]:
snowflake.set_environment(
    warehouse='COMPUTE_WH',
    role='ACCOUNTADMIN'
)

In [6]:
snowflake.database.create('NAAS', or_replace=True, silent=True)
snowflake.database.use('NAAS', silent=True)
snowflake.schema.create('NAAS_SCHEMA', or_replace=True, silent=True)
snowflake.schema.use('NAAS_SCHEMA', silent=True)

In [7]:
snowflake.get_environment()

{'role': 'ACCOUNTADMIN',
 'database': 'NAAS',
 'schema': 'NAAS_SCHEMA',
 'warehouse': 'COMPUTE_WH'}

## Output

### Creating file format

In [8]:
# It's possible to add extra parameters to the function
# They will be passed as additional variables at the end of the query statement
results_ff = snowflake.file_format.create(
    'my_json_format', 
    'JSON', 
    or_replace=True,
    TRIM_SPACE = 'TRUE'
)

results_ff

{'results': [('File format MY_JSON_FORMAT successfully created.',)],
 'description': [ResultMetadata(name='status', type_code=2, display_size=None, internal_size=16777216, precision=None, scale=None, is_nullable=True)],
 'statement': 'CREATE OR REPLACE FILE FORMAT my_json_format TYPE = JSON TRIM_SPACE = TRUE'}

In [9]:
# Let's not forget about `snowflake.execute` functionality
snowflake.execute('SHOW FILE FORMATS;')['results']

[(datetime.datetime(2022, 8, 6, 12, 2, 33, 880000, tzinfo=<DstTzInfo 'America/Los_Angeles' PDT-1 day, 17:00:00 DST>),
  'MY_JSON_FORMAT',
  'NAAS',
  'NAAS_SCHEMA',
  'JSON',
  'ACCOUNTADMIN',
  '',
  '{"TYPE":"JSON","FILE_EXTENSION":null,"DATE_FORMAT":"AUTO","TIME_FORMAT":"AUTO","TIMESTAMP_FORMAT":"AUTO","BINARY_FORMAT":"HEX","TRIM_SPACE":true,"NULL_IF":[],"COMPRESSION":"AUTO","ENABLE_OCTAL":false,"ALLOW_DUPLICATE":false,"STRIP_OUTER_ARRAY":false,"STRIP_NULL_VALUES":false,"IGNORE_UTF8_ERRORS":false,"REPLACE_INVALID_CHARACTERS":false,"SKIP_BYTE_ORDER_MARK":true}')]

### Creating file format with wrong parameter (not applicable for JSON file format, see [docs](https://docs.snowflake.com/en/sql-reference/sql/create-file-format.html))

In [10]:
try:
    snowflake.file_format.create('my_invalid_son_format', 'JSON', or_replace=True, ESCAPE = '"\\"')
except ProgrammingError as pe:
    print('Something went wrong!')
    print(pe)

Something went wrong!
002135 (42601): SQL compilation error:
Option ESCAPE is not valid for file format type JSON.


### Dropping file format

In [11]:
# snowflake.file_format.drop('my_json_format', if_exists=True)

### Creating internal stage

In [12]:
snowflake.stage.create(
    stage_name='my_internal_stage', 
    or_replace=True,
    file_format_name='my_json_format'
)

{'results': [('Stage area MY_INTERNAL_STAGE successfully created.',)],
 'description': [ResultMetadata(name='status', type_code=2, display_size=None, internal_size=16777216, precision=None, scale=None, is_nullable=True)],
 'statement': 'CREATE OR REPLACE STAGE my_internal_stage FILE_FORMAT = my_json_format'}

### Putting data on the stage

In [13]:
snowflake.stage.put(
    filepath='file://~/Downloads/reviews_data.json',
    internal_stage_name='@my_internal_stage',
    silent=True
)

### Listing files inside stage

In [14]:
result_list_stage = snowflake.stage.list('@my_internal_stage')

print(result_list_stage['results'])
print(result_list_stage['statement'])

[('my_internal_stage/reviews_data.json.gz', 538928, '6a141720c2d8dd29dc529b6b5d1995ba', 'Sat, 6 Aug 2022 19:02:37 GMT')]
LIST @my_internal_stage


### Creating table with query execution

In [15]:
query_create_table = """
    CREATE OR REPLACE TABLE reviews_dev (
      language VARCHAR
      , product_category VARCHAR
      , product_id VARCHAR
      , review_body VARCHAR
      , review_id VARCHAR
      , review_title VARCHAR
      , reviewer_id VARCHAR
      , stars INT
    );
"""

# No worries, Table API will be available soon too!
snowflake.execute(query_create_table)['results']

[('Table REVIEWS_DEV successfully created.',)]

### Loading data from internal stage to a table

As we're loading data in JSON format, transformations are required to not put everything into a single VARIANT type column (for more, [see the documentation](https://docs.snowflake.com/en/sql-reference/data-types-semistructured.html)).

In [16]:
transformation_statement = """
    SELECT
        $1:language::varchar AS language
        , $1:product_category::varchar AS product_category
        , $1:product_id::varchar AS product_id
        , $1:review_body::varchar AS review_body
        , $1:review_id::varchar AS review_id
        , $1:review_title::varchar AS review_title
        , $1:reviewer_id::varchar AS reviewer_id
        , $1:stars::int AS stars
    FROM @my_internal_stage
"""

snowflake.copy_into(
    table_name='reviews_dev',
    source_stage=transformation_statement,
    silent=True
)

### Data querying straight to Pandas DataFrame

In [17]:
reviews_dev = snowflake.query_pd('SELECT * FROM reviews_dev')
reviews_dev.head()

,LANGUAGE,PRODUCT_CATEGORY,PRODUCT_ID,REVIEW_BODY,REVIEW_ID,REVIEW_TITLE,REVIEWER_ID,STARS
0,en,baby_product,product_en_0878845,Pathetic design of the caps. Very impractical ...,en_0968227,Not worth the price and very bad cap design,reviewer_en_0987470,1
1,en,shoes,product_en_0004522,"Shoes were purchased on March 6, 2019. My wife...",en_0830781,Garbage!,reviewer_en_0731158,1
2,en,office_product,product_en_0060687,It's taken me 1 whole year to set this thing u...,en_0277954,I do not recommend this printer,reviewer_en_0793876,1
3,en,office_product,product_en_0311791,Each cartridge printed once. Both dried up in ...,en_0316499,Don't purchase these refurbished cartridges!,reviewer_en_0837288,1
4,en,baby_product,product_en_0472877,No light hard to see,en_0320665,Not worth,reviewer_en_0878169,1
